In [1]:
import os
import numpy as np
import gdown
from pathlib import Path

import tensorflow as tf
from tensorflow import keras

In [2]:
url_path_map = {
        'train': {
            'url':'https://drive.google.com/uc?id=1JewlmOsqs-O1EmMMKutcYYiD7GkN6brN', 
            'path':'dataset/train.npy'
            },
        'test': {
            'url':'https://drive.google.com/uc?id=1JcCFZbc_N7VIa3G3XA8grwz4XGdaaXSZ', 
            'path':'dataset/test.npy'
            },
        'label': {
            'url':'https://drive.google.com/uc?id=1JiaTOcZ6QfDThw3RrIa3ZxDDaNt-BMUQ',
            'path':'dataset/label_info.txt'
        }
    }

In [3]:
def download_dataset():
    Path('dataset').mkdir(exist_ok=True)
    for split in ['train', 'test', 'label']:
        if Path(url_path_map[split]['path']).exists():
            continue
        gdown.download(url_path_map[split]['url'], url_path_map[split]['path'], quiet=False)

In [4]:
download_dataset()

Downloading...
From: https://drive.google.com/uc?id=1JewlmOsqs-O1EmMMKutcYYiD7GkN6brN
To: /content/dataset/train.npy
753MB [00:02, 332MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JcCFZbc_N7VIa3G3XA8grwz4XGdaaXSZ
To: /content/dataset/test.npy
753MB [00:02, 279MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JiaTOcZ6QfDThw3RrIa3ZxDDaNt-BMUQ
To: /content/dataset/label_info.txt
100%|██████████| 534/534 [00:00<00:00, 672kB/s]


In [5]:
def read_txt(file_name):
    with open(file_name, 'rt') as f:
        class_list = [row.strip('\n') for row in f.readlines()]
    return class_list

In [6]:
train_dir = '/content/dataset/train.npy'
test_dir = '/content/dataset/test.npy'
label_dir = '/content/dataset/label_info.txt'

In [7]:
train = np.load(train_dir, allow_pickle=True).item()
test = np.load(test_dir, allow_pickle=True).item()

In [8]:
train_image , train_label = train['image'], train['label']
test_image, test_label = test['image'], test['label']

In [9]:
train_image.shape, train_label.shape

((500, 224, 224, 30), (500,))

In [10]:
test_image.shape, test_label.shape

((500, 224, 224, 30), (500,))

In [11]:
train_image = np.split(train_image, 10, axis=3)
test_image = np.split(test_image, 10, axis=3)

In [12]:
train_image = np.array(train_image)
test_image = np.array(test_image)

In [13]:
train_image.shape, test_image.shape

((10, 500, 224, 224, 3), (10, 500, 224, 224, 3))

In [14]:
train_image = train_image.reshape((-1, 224, 224, 3))
test_image = test_image.reshape((-1, 224, 224, 3))

In [15]:
train_image.shape, test_image.shape

((5000, 224, 224, 3), (5000, 224, 224, 3))

In [16]:
label_temp = np.append(train_label, train_label) #2
label_temp = np.append(label_temp, label_temp) #4
label_temp = np.append(label_temp, label_temp) #8
label_temp = np.append(label_temp, train_label) #9
train_label_extended = np.append(label_temp, train_label) #10

In [17]:
test_label_extended = np.zeros(shape=(5000,))

In [18]:
train_label_extended.shape

(5000,)

In [19]:
train_image = train_image.astype('float32') / 255.0
test_image = test_image.astype('float32') / 255.0

In [20]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label_extended))
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_label_extended))

In [21]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 5000

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [22]:
import matplotlib.pyplot as plt
'''
#plt.figure(figsize=(3, 10))
for i in range(30):
  plt.subplot(3, 10, i+1)
  plt.imshow(train_image[i])
  '''

'\n#plt.figure(figsize=(3, 10))\nfor i in range(30):\n  plt.subplot(3, 10, i+1)\n  plt.imshow(train_image[i])\n  '

In [23]:
#plt.imshow(train_image[54])

In [24]:
label_list = read_txt(label_dir)

In [25]:
label_list

['Bergamot',
 'BlanketFlower',
 'BleedingHeart',
 'Bletilla',
 'Bluebonnets',
 'BluestarFlower',
 'Brassica',
 'Broom',
 'Calceolaria',
 'Celosia',
 'Chionodoxa',
 'CoralBells',
 'CuckooFlower',
 'Dietes',
 'DutchIris',
 'EnglishBluebell',
 'Eremurus',
 'FrangipaniFlower',
 'Fuchsia',
 'Guzmania',
 'Hebe',
 'Helenium',
 'Hellebore',
 'Honeysuckle',
 'Ixia',
 'Jacob’sLadder',
 'KangarooPaw',
 'Knautia',
 'Lewesia',
 'Lotus',
 'Meconopsis',
 'Mimosa',
 'MorningGlory',
 'NewZealandTeaTree',
 'OrientalLily',
 'Oxalis',
 'OysterPlant',
 'Pansy',
 'Photinia',
 'PincushionFlower',
 'Poinsettia',
 'Queen’sCup',
 'RockRose',
 'Scaevola',
 'ScentedGeranium',
 'Statice',
 'Tuberose',
 'UrnPlant',
 'VirginiaCreeper',
 'Winterberry']

In [26]:
from tensorflow.keras import models, layers

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D

In [28]:
model = models.Sequential()

In [29]:
# 1st Conv2d
#  - filter_cnt : 32 - kerner_size : (3,3) - relu
model.add(layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                        input_shape=(224, 224, 3),
                        name='block_1_conv'))
# 1st max pooling
# - pool_size = (2,2)
model.add(layers.MaxPool2D(pool_size=2,name='block_1_pool'))
# 2nd Conv2d
#  - filter_cnt : 64 - kerner_size : (3,3) - relu
model.add(layers.Conv2D(filters=64, kernel_size=3, activation='relu',
                        name='block_2_conv'))
# 2nd max pooling
# - pool_size = (2,2)
model.add(layers.MaxPool2D(pool_size=2, name='block_2_pool'))
# 3rd Conv2d
#  - filter_cnt : 64 - kerner_size : (3,3) - relu
model.add(layers.Conv2D(filters=64, kernel_size=3, activation='relu',
                        name='block_3_conv'))
# Flatten
model.add(layers.Flatten())

#classification module
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=50, activation='softmax'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block_1_conv (Conv2D)        (None, 222, 222, 32)      896       
_________________________________________________________________
block_1_pool (MaxPooling2D)  (None, 111, 111, 32)      0         
_________________________________________________________________
block_2_conv (Conv2D)        (None, 109, 109, 64)      18496     
_________________________________________________________________
block_2_pool (MaxPooling2D)  (None, 54, 54, 64)        0         
_________________________________________________________________
block_3_conv (Conv2D)        (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 173056)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

In [31]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [33]:
model.fit(train_dataset, epochs=30)

Epoch 1/30
79/79 [==============================] - 5s 57ms/step - loss: 3.3655 - accuracy: 0.1568
Epoch 2/30
79/79 [==============================] - 5s 57ms/step - loss: 2.6760 - accuracy: 0.3288
Epoch 3/30
79/79 [==============================] - 4s 57ms/step - loss: 1.7648 - accuracy: 0.5816
Epoch 4/30
79/79 [==============================] - 5s 57ms/step - loss: 1.2270 - accuracy: 0.7418
Epoch 5/30
79/79 [==============================] - 5s 57ms/step - loss: 0.9509 - accuracy: 0.7962
Epoch 6/30
79/79 [==============================] - 5s 57ms/step - loss: 0.7726 - accuracy: 0.8320
Epoch 7/30
79/79 [==============================] - 5s 57ms/step - loss: 0.7165 - accuracy: 0.8384
Epoch 8/30
79/79 [==============================] - 5s 57ms/step - loss: 0.6173 - accuracy: 0.8468
Epoch 9/30
79/79 [==============================] - 5s 57ms/step - loss: 0.5790 - accuracy: 0.8480
Epoch 10/30
79/79 [==============================] - 4s 57ms/step - loss: 0.5422 - accuracy: 0.8558
Epoch 11/

In [34]:
loss, accuracy = model.evaluate(train_dataset)

79/79 [==============================] - 5s 30ms/step - loss: 0.2418 - accuracy: 0.8924


In [35]:
predictions = model.predict(test_image)

In [36]:
np.argmax(predictions[2])

41

In [37]:
import pandas as pd

In [38]:
id=[]
pred=[]
for i in range(500):
  id.append(i)
  pred.append(np.argmax(predictions[i]))

In [39]:
challenge = pd.DataFrame({'image_id':id, 'label':pred})

In [40]:
challenge

,image_id,label
0,0,15
1,1,36
2,2,41
3,3,6
4,4,32
...,...,...
495,495,28
496,496,19
497,497,18
498,498,26


In [41]:
challenge.to_csv('deepChallenge05.csv', index=False)